<a href="https://colab.research.google.com/github/Vvalentinaa/N-gram-Language-Model/blob/main/N_gram_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\textbf{Реализация классической N-gram Language Model} \\
\textbf{0. Библиотеки}$
  *   **re** - $\text{для работы с регулярными выражениями}$
  *   **nltk** - $\text{пакет библиотек и программ для символьной и статистической обработки естественного языка}$



In [ ]:
import re
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk import bigrams, trigrams, ngrams
from nltk import word_tokenize
from nltk.lm.preprocessing import pad_both_ends
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

$\text{Добавим в дефолтный список пунктуации еще символы,} \\ \text{ которые специфичны для данного текста, в частности:} \\ \text{троеточие, 11 символов "-" для обозначения сноски:}$

In [ ]:
punctuation += '``...-----------'

$\textbf{1. Импорт и препроцессинг данных}$

  1.1   $\text{Импорт данных}$  
  
  1.2    $\text{Препроцессинг - токенизация, разбиение на n-граммы}$ 

$\text{Данная реализация выполнена с помощью среды Google Colab,поэтому данные импортируются непосредственно с пользовательского гугл-диска.}
\\ 
\text{Выполняя код на локальном блокноте Jupyter Notebook, не составит труда прописать путь до данных на своем ПК.}$




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/tolstoi_childhood.txt'
textfile = open(path, encoding='utf-8-sig').read()  
textfile = re.sub(r'<.+>', '', textfile)      # удаление html-тегов
textfile = re.sub(r'\n', ' ', textfile)       # замена символа окончания строки \n на пробел
sents = nltk.sent_tokenize(textfile.lower())  # разбиение текста на предложения
tokens = []
for sent in sents:
  sent = nltk.word_tokenize(sent)             # разбиение предложений на слова
  sent = [word for word in sent if word not in punctuation]   # удаление пунктуации
  tokens.append(sent)

$
\text{Хотелось бы еще знать, какие слова чаще встречаются в начале предложения или в конце. Для решения этой задачи в библиотеке nltk } \\ \text{ есть функция pad_sequence, добавляющая задаваемый пользователем символ в начало и в конец предложения(опционально).}  \\
\text{Для удобства будем использовать специальную функцию в модуле nltk.lm - pad_both_ends, добавляющая спецсимвол <s> } \\ \text{в начало предложения и аналогично <\s> в конец.}
$


In [ ]:
trigram_tokens = []
for i in range(len(tokens)):
  trigram_tokens.append(list(trigrams(pad_both_ends(tokens[i], n=2)))) 
trigram_tokens[0]

In [ ]:
# УБРАТЬ !!!!!
from nltk.util import everygrams
padded_trigrams = []
for i in range(len(tokens)):
  padded_trigrams.append(list(pad_both_ends(tokens[i], n=3)))
list(everygrams(padded_trigrams, max_len=3))

$\text{Теперь создадим "словарик", чтобы модель знала, какие слова ей известны.}$

In [ ]:
from nltk.lm.preprocessing import padded_everygram_pipeline
train, vocab = padded_everygram_pipeline(2, tokens)

In [ ]:
list(trigrams(pad_both_ends(tokens[0], n=3)))

$\textbf{2. Training}$
$
\text{Библиотека nltk содержит семейство языковых моделей, которые считают условные вероятности при помощи частотного анализа появления} \\ \text{ в тексте n-грам.} \\ 
\text{Один из таких - класс MLE (Maximum Likelihood Estimator), построенный на основе на основе метода максимального правдоподобия.}
$

In [ ]:
from nltk.lm import MLE
lm = MLE(3)
lm.fit(train, vocab)  # обучение

In [ ]:
print(lm.counts)

<NgramCounter with 2 ngram orders and 1359976 ngrams>


In [ ]:
# lm.counts[['карл']]['иваныч']
w = "карл"
print(lm.counts[w])
print(lm.score(w))
print(lm.score("иваныч", ["карл"]))

90
0.0001285326851477626
0.8333333333333334


In [ ]:
lm.generate(6, random_seed=0)

['собой', 'разумеется', 'не', 'желающие', 'казаться', 'логическим']

In [ ]:
lm.generate(8, text_seed=['карл'])

['иваныч', 'во-первых', 'чтобы', 'народы', 'и', 'что', 'он', 'не']

$\textbf{3. Сохранение модели}$


In [ ]:
import pickle
with open('ngram_language_model.pkl', 'wb') as fout:
    pickle.dump(lm, fout)


with open('ngram_language_model.pkl', 'rb') as fin:
    model_loaded = pickle.load(fin)

lm.generate(20)


['вслед',
 'за',
 'лекарем',
 '</s>',
 'пьер',
 'должен',
 'был',
 'принят',
 'был',
 'большой',
 'ни',
 'бранить',
 'ни',
 'старался',
 'кутузов',
 'глядел',
 'на',
 'его',
 'воображении',
 'с']